In [57]:
import os
import pandas as pd

# Folder containing daily odds CSV files
folder_path = "daily_odds"

# Set to store unique pitcher names
unique_pitchers = set()

# Iterate through all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_csv(file_path)
            if 'homePitcher' in df.columns and 'awayPitcher' in df.columns:
                unique_pitchers.update(df['homePitcher'].dropna().unique())
                unique_pitchers.update(df['awayPitcher'].dropna().unique())
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Convert to sorted list for presentation
unique_pitchers = sorted(unique_pitchers)
pitcher_df = pd.DataFrame(unique_pitchers, columns=["Pitcher Name"])

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

# Original pitcher names
names = list(pitcher_df['Pitcher Name'])
normalized_names = [name.lower().replace(" ", "") for name in names]

# Store results and failures
all_players = []
failed_lookups = []

# Loop with tqdm progress bar
for original_name, normalized_name in tqdm(zip(names, normalized_names), total=len(names), desc="Looking up pitchers"):
    url = f"https://statsapi.mlb.com/api/v1/people/search?names={normalized_name}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        players = data.get("people", [])
        
        if players:
            for player in players:
                player["input_name"] = original_name
                all_players.append(player)
        else:
            failed_lookups.append(original_name)
    except Exception as e:
        failed_lookups.append(original_name)

# Convert results to DataFrame
df_players = pd.DataFrame(all_players)

# Show relevant columns
if not df_players.empty:
    print(df_players[["id", "fullName", "input_name"]])

# Report failures
if failed_lookups:
    print("\n❌ Names that returned no results:")
    for name in failed_lookups:
        print(f" - {name}")

Looking up pitchers: 100%|██████████| 527/527 [03:50<00:00,  2.29it/s]

         id          fullName        input_name
0    700363  AJ Smith-Shawver  AJ Smith-Shawver
1    676879       Aaron Ashby       Aaron Ashby
2    605156      Aaron Brooks      Aaron Brooks
3    650644      Aaron Civale      Aaron Civale
4    605400        Aaron Nola        Aaron Nola
..      ...               ...               ...
523  677161        Zack Kelly        Zack Kelly
524  641793      Zack Littell      Zack Littell
525  668868     Zack Thompson     Zack Thompson
526  554430      Zack Wheeler      Zack Wheeler
527  805673    Zebby Matthews    Zebby Matthews

[528 rows x 3 columns]

❌ Names that returned no results:
 - Alexander Cobb
 - Andrew Hutchison
 - Chase Gockel
 - Christoper Flexen
 - Christopher Archer
 - David Knehr
 - Donald Greinke
 - Frankie Montas Jr.
 - Frederic Tarnok
 - Jacob Lugo
 - James Lambert
 - James Yacabonis
 - John Ober
 - John Patrick Sears
 - Joseph Mantiply
 - Joseph Wentz
 - Joshua Staumont
 - Kenneth Kelly
 - Kenneth Rosenberg
 - Kirkland McCar

In [40]:
df_clean = df_players[["id", "fullName"]]
df_clean

,id,fullName
0,700363,AJ Smith-Shawver
1,676879,Aaron Ashby
2,605156,Aaron Brooks
3,650644,Aaron Civale
4,605400,Aaron Nola
...,...,...
523,677161,Zack Kelly
524,641793,Zack Littell
525,668868,Zack Thompson
526,554430,Zack Wheeler


In [41]:
manual_entries = [
    ("Alexander Cobb", 502171),
    ("Andrew Hutchison", 571800),
    ("Chase Gockel", 813905),
    ("Christoper Flexen", 623167),
    ("Christopher Archer", 502042),
    ("David Knehr", 663753),
    ("Donald Greinke", 425844),
    ("Frankie Montas Jr.", 593423),
    ("Frederic Tarnok", 676206),
    ("Jacob Lugo", 607625),
    ("James Lambert", 669424),
    ("James Yacabonis", 642231),
    ("John Ober", 641927),
    ("John Patrick Sears", 676664),
    ("Joseph Mantiply", 573009),
    ("Joseph Wentz", 666214),
    ("Joshua Staumont", 622251),
    ("Kenneth Kelly", 220305),
    ("Kenneth Rosenberg", 670046),
    ("Kirkland McCarty", 668948),
    ("Matthew Bush", 456713),
    ("Matthew Dermody", 571616),
    ("Matthew Swarmer", 664161),
    ("Matthew Wisler", 605538),
    ("Maxwell Scherzer", 453286),
    ("Michael Lynn", 543477),
    ("Michael Minor", 501985),
    ("Neil Pallante", 669467),
    ("Nicholas Neidert", 663734),
    ("Patrick Naughton", 676050),
    ("Raymond Kerr", 678061),
    ("Timothy James Zeuch", 643615),
    ("Todd Smyly", 592767),
    ("Tomaso Milone", 543548),
    ("Zachary Eflin", 621107),
    ("Zachary Wheeler", 554430),
]

# Convert to DataFrame
df_manual = pd.DataFrame(manual_entries, columns=["fullName", "id"])

# Reorder columns to match df_clean
df_manual = df_manual[["id", "fullName"]]

# Append to the clean DataFrame
df_all = pd.concat([df_clean, df_manual], ignore_index=True)

# Optional: save or view
# df_all.to_csv("all_pitcher_ids.csv", index=False)
print(df_all.tail())


         id             fullName
559  643615  Timothy James Zeuch
560  592767           Todd Smyly
561  543548        Tomaso Milone
562  621107        Zachary Eflin
563  554430      Zachary Wheeler


In [59]:
df_all

,id,fullName
0,700363,AJ Smith-Shawver
1,676879,Aaron Ashby
2,605156,Aaron Brooks
3,650644,Aaron Civale
4,605400,Aaron Nola
...,...,...
559,643615,Timothy James Zeuch
560,592767,Todd Smyly
561,543548,Tomaso Milone
562,621107,Zachary Eflin


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Setup headless Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(options=options)

# Pitcher info
player_id = 666120
year = 2021
url = f"https://www.mlb.com/player/ian-anderson-{player_id}?stats=gamelogs-r-pitching-mlb&year={year}"

# Load page
driver.get(url)
time.sleep(5)  # Let JS content load
a
# Grab HTML
html = driver.page_source

# Save to file for inspection
with open(f"ian_anderson_{year}_gamelog.html", "w", encoding="utf-8") as f:
    f.write(html)

driver.quit()
print(f"✅ Saved raw HTML for Ian Anderson {year} to file.")


✅ Saved raw HTML for Ian Anderson 2021 to file.


In [56]:
import re
import pandas as pd

# Load HTML
with open("ian_anderson_2021_gamelog.html", "r", encoding="utf-8") as f:
    html = f.read()

# Get all rows
row_pattern = re.compile(r'<tr data-index="\d+">(.*?)</tr>', re.DOTALL)
rows = row_pattern.findall(html)

# Column extractor
def extract_col(row_html, col_index):
    if col_index == 0:
        match = re.search(rf'data-col="{col_index}".*?<a[^>]*>([^<]+)</a>', row_html, re.DOTALL)
    else:
        match = re.search(rf'data-col="{col_index}".*?<span>([^<]+)</span>', row_html, re.DOTALL)
    return match.group(1).strip() if match else None

# Filtered row extraction
data = []
for row in rows:
    date = extract_col(row, 0)
    team = extract_col(row, 1)
    opponent = extract_col(row, 2)
    era = extract_col(row, 5)

    # Only include if team looks like a valid abbreviation
    if team and re.fullmatch(r"[A-Z]{3}", team):
        data.append((date, team, opponent, era))

# Create DataFrame
df = pd.DataFrame(data, columns=["Date", "Team", "Opponent", "ERA"])
df


,Date,Team,Opponent,ERA
0,Apr 4,ATL,@ PHI,1.80
1,Apr 10,ATL,vs PHI,4.35
2,Apr 15,ATL,vs MIA,4.70
3,Apr 21,ATL,@ NYY,3.27
4,Apr 27,ATL,vs CHC,2.48
5,May 2,ATL,@ TOR,3.27
6,May 8,ATL,vs PHI,3.46
7,May 15,ATL,@ MIL,3.20
8,May 21,ATL,vs PIT,2.82
9,May 29,ATL,@ NYM,3.27


In [58]:
import os
import pandas as pd

# Folder with your CSVs
folder_path = "daily_odds"

# Store results as list of dicts
pitcher_years = []

# Process each file
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_csv(file_path)

            # Ensure required columns exist
            if 'homePitcher' in df.columns and 'awayPitcher' in df.columns and 'startDate' in df.columns:
                # Extract year from ISO timestamp
                df['year'] = pd.to_datetime(df['startDate']).dt.year

                # Collect pitcher name and year
                for _, row in df.iterrows():
                    if pd.notna(row['homePitcher']):
                        pitcher_years.append({"Pitcher Name": row['homePitcher'], "Year": row['year']})
                    if pd.notna(row['awayPitcher']):
                        pitcher_years.append({"Pitcher Name": row['awayPitcher'], "Year": row['year']})
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Convert to DataFrame
pitcher_year_df = pd.DataFrame(pitcher_years).drop_duplicates().sort_values(by=["Pitcher Name", "Year"])

pitcher_year_df

,Pitcher Name,Year
1538,AJ Smith-Shawver,2023
3877,AJ Smith-Shawver,2024
165,Aaron Ashby,2022
5937,Aaron Ashby,2024
2872,Aaron Brooks,2024
...,...,...
1476,Zack Thompson,2023
507,Zack Wheeler,2022
58,Zack Wheeler,2023
284,Zack Wheeler,2024


In [64]:
# First, make sure the name columns match
df_all = df_all.rename(columns={"fullName": "Pitcher Name"})

# Merge on Pitcher Name
merged_df = pitcher_year_df.merge(df_all, on="Pitcher Name", how="left")

# Reorder columns if desired
merged_df = merged_df[["Pitcher Name", "id", "Year"]]
# Attempt to convert only non-NaN values to int (optional if needed for downstream use)
merged_df['id'] = pd.to_numeric(merged_df['id'], errors='coerce')

# Identify unmatched pitcher names
unmatched = merged_df[merged_df['id'].isna()]['Pitcher Name'].unique()



In [73]:
# Manually scraped name-ID pairs
manual_ids = {
    "Adrian Martinez": 661309,
    "Anibal Sanchez": 434671,
    "Carlos Rodon": 607074,
    "Chi Chi Gonzalez": 592346,
    "Cristopher Sanchez": 650911,
    "Domingo German": 593334,
    "Elieser Hernandez": 622694,
    "Emilio Pagan": 641941,
    "Eury Perez": 641941,
    "Francisco Perez": 660906,
    "German Marquez": 608566,
    "Jesus Luzardo": 666200,
    "Jhonathan Diaz": 646242,
    "Jose Berrios": 621244,
    "Jose De Leon": 592254,
    "Jose Soriano": 667755,
    "Jose Urena": 570632,
    "Jose Urquidy": 664353,
    "José Quintana": 500779,
    "Julio Urias": 628711,
    "Luis Patino": 672715,
    "Martin Perez": 527048,
    "Pablo Lopez": 641154,
    "Randy Rodriguez": 678495,
    "Ranger Suarez": 624133,
    "Reynaldo Lopez": 625643,
    "Robert Wood": 124541,
    "Roddery Munoz": 682610,
    "Rony Garcia": 665621,
    "Sixto Sanchez": 664350,
    "Victor Gonzalez": 624647,
    "Yariel Rodriguez": 684320,
    "Yerry Rodriguez": 666720,
    "Yilber Diaz": 700270
}

# Only update rows where ID is missing and name is in the manual list
for name, player_id in manual_ids.items():
    mask = (merged_df['Pitcher Name'] == name) & (merged_df['id'].isna())
    merged_df.loc[mask, 'id'] = player_id

# Optional: cast to int (if you want a clean numeric type now)
merged_df['id'] = merged_df['id'].astype('Int64')  # pandas nullable int type

# ✅ Done
print("Manual IDs patched.")


Manual IDs patched.


In [75]:
merged_df

,Pitcher Name,id,Year
0,AJ Smith-Shawver,700363,2023
1,AJ Smith-Shawver,700363,2024
2,Aaron Ashby,676879,2022
3,Aaron Ashby,676879,2024
4,Aaron Brooks,605156,2024
...,...,...,...
941,Zack Thompson,668868,2023
942,Zack Wheeler,554430,2022
943,Zack Wheeler,554430,2023
944,Zack Wheeler,554430,2024


In [ ]:
import os
import time
import re
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Ensure output folder exists
output_dir = "test_data/raw_pitcher_data"
os.makedirs(output_dir, exist_ok=True)

# List to track errors
failed_downloads = []

# Setup Selenium headless browser
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
)
driver = webdriver.Chrome(options=options)

# Loop with progress bar
for idx, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc="Scraping pitchers"):
    name = row["Pitcher Name"]
    player_id = row["id"]
    year = row["Year"]

    if pd.isna(name) or pd.isna(player_id) or pd.isna(year):
        continue

    # Format name: lowercase, spaces to hyphens
    url_name = re.sub(r"\s+", "-", name.strip().lower())

    url = f"https://www.mlb.com/player/{url_name}-{int(player_id)}?stats=gamelogs-r-pitching-mlb&year={int(year)}"

    try:
        driver.get(url)
        time.sleep(5)  # Let the page load
        html = driver.page_source

        filename = f"{url_name}_{int(year)}.html"
        filepath = os.path.join(output_dir, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(html)

    except Exception as e:
        failed_downloads.append(f"{name} ({year})")
        continue

# Clean up
driver.quit()

# Report
if failed_downloads:
    print("\n⚠️ Failed Downloads:")
    for entry in failed_downloads:
        print(f" - {entry}")
else:
    print("\n✅ All pitcher HTML files successfully saved.")


Scraping pitchers: 100%|██████████| 946/946 [1:41:13<00:00,  6.42s/it]



✅ All pitcher HTML files successfully saved.


In [83]:
import os
import re
import pandas as pd

def extract_pitcher_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        html = f.read()

    filename = os.path.basename(file_path)
    name, year = filename.replace(".html", "").rsplit("_", 1)
    player_name = name.replace("-", " ").title()  # Convert back to "Firstname Lastname"

    row_pattern = re.compile(r'<tr data-index="\d+">(.*?)</tr>', re.DOTALL)
    rows = row_pattern.findall(html)

    def extract_col(row_html, col_index):
        if col_index == 0:
            match = re.search(rf'data-col="{col_index}".*?<a[^>]*>([^<]+)</a>', row_html, re.DOTALL)
        else:
            match = re.search(rf'data-col="{col_index}".*?<span>([^<]+)</span>', row_html, re.DOTALL)
        return match.group(1).strip() if match else None

    data = []
    for row in rows:
        date = extract_col(row, 0)
        team = extract_col(row, 1)
        opponent = extract_col(row, 2)
        era = extract_col(row, 5)

        if team and re.fullmatch(r"[A-Z]{3}", team):
            data.append((date, team, opponent, era, player_name, year))

    return pd.DataFrame(data, columns=["Date", "Team", "Opponent", "ERA", "Player", "Year"])

# Automatically grab first HTML file
folder = "raw_pitcher_data"
files = sorted(f for f in os.listdir(folder) if f.endswith(".html"))
first_file_path = os.path.join(folder, files[500])

# Extract data
df = extract_pitcher_data(first_file_path)
print(f"✅ Extracted from: {first_file_path}")
df

✅ Extracted from: raw_pitcher_data/kodai-senga_2024.html


,Date,Team,Opponent,ERA,Player,Year
0,Jul 26,NYM,vs ATL,3.38,Kodai Senga,2024
1,Jul 26,NYM,vs ATL,3.38,Kodai Senga,2024


In [84]:
import os
import re
import pandas as pd
from tqdm import tqdm

def extract_pitcher_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        html = f.read()

    filename = os.path.basename(file_path)
    name, year = filename.replace(".html", "").rsplit("_", 1)
    player_name = name.replace("-", " ").title()

    row_pattern = re.compile(r'<tr data-index="\d+">(.*?)</tr>', re.DOTALL)
    rows = row_pattern.findall(html)

    def extract_col(row_html, col_index):
        if col_index == 0:
            match = re.search(rf'data-col="{col_index}".*?<a[^>]*>([^<]+)</a>', row_html, re.DOTALL)
        else:
            match = re.search(rf'data-col="{col_index}".*?<span>([^<]+)</span>', row_html, re.DOTALL)
        return match.group(1).strip() if match else None

    data = []
    for row in rows:
        date = extract_col(row, 0)
        team = extract_col(row, 1)
        opponent = extract_col(row, 2)
        era = extract_col(row, 5)
        if team and re.fullmatch(r"[A-Z]{3}", team):
            data.append((date, team, opponent, era, player_name, year))

    return pd.DataFrame(data, columns=["Date", "Team", "Opponent", "ERA", "Player", "Year"])

# Folder and file setup
folder = "raw_pitcher_data"
all_dfs = []

# Progress bar
for file in tqdm(sorted(os.listdir(folder))):
    if file.endswith(".html"):
        path = os.path.join(folder, file)
        try:
            df = extract_pitcher_data(path)
            all_dfs.append(df)
        except Exception as e:
            print(f"❌ Failed to process {file}: {e}")

# Combine and save
final_df = pd.concat(all_dfs, ignore_index=True)
final_df.to_csv("combined_pitcher_gamelogs.csv", index=False)
print("✅ Saved to combined_pitcher_gamelogs.csv")


100%|██████████| 900/900 [00:03<00:00, 265.49it/s]

✅ Saved to combined_pitcher_gamelogs.csv
